# Сессия 3

## 3.0 Загрузка данных

In [1]:
import pickle

In [2]:
data = pickle.load(open('splitted_data.pkl', 'rb'))

In [3]:
data = {'x_tr' : data[0], 'x_te': data[1], 'y_tr': data[2], 'y_te': data[3]}

In [4]:
data['x_tr'] = data['x_tr'].drop(columns='Должность_автора')
data['x_te'] = data['x_te'].drop(columns='Должность_автора')

## 3.1 Обучение

По причинам рассмотренными в предыдущей сессии для обучения буду брать модели MultinomialNB, ComplementNB и RandomForestClassifier

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier

В качестве метрик буду использовать f1_score, которая является общей метрикой определения ошибок модели; а также метрику точности модели, выявляющую количество правильных предсказаний делённое на общее количество предсказаний

In [6]:
from sklearn.metrics import f1_score, accuracy_score

MultinomialNB

In [10]:
MNB = MultinomialNB().fit(data['x_tr'], data['y_tr'])

In [11]:
f1_score(MNB.predict(data['x_te']), data['y_te'], average='weighted')

0.6548631767017012

In [12]:
accuracy_score(MNB.predict(data['x_te']), data['y_te'])

0.623030303030303

ComplementNB

In [13]:
CNB = ComplementNB().fit(data['x_tr'], data['y_tr'])

In [14]:
f1_score(CNB.predict(data['x_te']), data['y_te'], average='weighted')

0.6717248557870842

In [15]:
accuracy_score(CNB.predict(data['x_te']), data['y_te'])

0.6315151515151515

RandomForestClassifier

In [22]:
RFC = RandomForestClassifier(random_state=1).fit(data['x_tr'], data['y_tr'])

In [23]:
f1_score(RFC.predict(data['x_te']), data['y_te'], average='weighted')

0.6744430142513936

In [24]:
accuracy_score(RFC.predict(data['x_te']), data['y_te'])

0.6593939393939394

Наилучшим образом показала себя модель RandomForestClassifier, при этом обучаясь не на много дольше остпльных моделей 

Её и буду использовать для дальнейшего обучения и проверок

## 3.2 Feature Engineering

В качестве Feature Engineering попробую удалить некоторые малоинформативные атрибуты, что позволит избавиться от лишних данных; а также удалю редкие классы, шансы предсказания которых равны или близки к нулю. Это позволит сократить количество данных и в теории улучшить точность модели

Для простоты преобразований соединю пока весь разделённый датасет в один

In [25]:
import pandas as pd

In [26]:
data = pd.concat([
    pd.concat([data['y_tr'], data['x_tr']], axis = 1),
    pd.concat([data['y_te'], data['x_te']], axis = 1)
])

In [27]:
data

,Должность_адресата,cad,cis,competition,digitallskills,digitalskills,esim,europe,euroskills,futureskills,...,электронный,энергия,этап,этот,юниор,якутия,якутск,ялтинский,янв,январь
4309,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4754,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2370,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
225,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5269,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2814,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2029,Ассоциация участников технологических кружков,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5413,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.2.1 Удаление лишних атрибутов

За лишние атрибуты можно принять слова, которые не несут информации, по типу "вх", "на", "от", а также очень редкие атрибуты, суммарное количество появления которых меньше 10 раз

In [28]:
key_words = data.drop(columns='Должность_адресата').sum().sort_values()

In [29]:
key_words[key_words < 10]

дигиталскиллс                      3
всероссийскуюнаучнопрактический    5
digitallskills                     5
взаимный                           5
крым                               5
                                  ..
далее                              9
ава                                9
инновационный                      9
главный                            9
гарантийный                        9
Length: 261, dtype: int64

In [30]:
attrs_to_del = list(key_words[key_words < 10].index)

In [31]:
key_words[-50:]

экзамен              213
рамка                216
участник             217
технология           219
союз                 219
согласование         220
рф                   222
республика           231
информация           234
организация          242
рабочий              244
перечень             247
предоставление       249
поручение            249
мероприятие          251
организационный      253
совет                260
приглашение          265
соревнование         278
финал                291
отборочный           293
область              307
казань               311
мировой              320
мастерство           327
от                   329
региональный         362
заявка               377
профессионал         386
молодой              391
заседание            397
программа            428
подготовка           436
проект               441
национальный         442
профессиональный     460
компетенция          495
россия               544
для                  555
worldskills          578


In [32]:
attrs_to_del.extend('''от для на по вх'''.split())

In [33]:
data = data.drop(columns=attrs_to_del)

### 3.2.2 Удаление редких классов

За редкие классы можно принять классы с частотой появления меньше 10 записей 

In [34]:
classes_to_del = data['Должность_адресата'].value_counts()[data['Должность_адресата'].value_counts() < 10].index

Записей с редкими классами суммарно всего 355, что является небольшим количеством относительно общего количества данных, при этом позволяя достаточно хорошо очистить данные

In [35]:
data['Должность_адресата'].isin(classes_to_del).sum()

355

Удаляю сами записи 

In [36]:
data = data[~data['Должность_адресата'].isin(classes_to_del)]

In [42]:
data

,Должность_адресата,cis,competition,digitalskills,esim,euroskills,futureskills,hitech,ii,iii,...,электромонтаж,электронный,энергия,этап,этот,юниор,якутия,якутск,янв,январь
4309,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4754,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2370,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
225,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3062,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5576,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5269,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2814,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5413,Агентство развития профессиональных сообществ ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.2.3 Разбиение данных и проверка модели

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns='Должность_адресата'), data['Должность_адресата'], stratify=data['Должность_адресата'], test_size=0.15)

Проверка RandomForestClassifier с такими же параметрами и метриками качества обучения

In [45]:
RFC1 = RandomForestClassifier(random_state=1).fit(x_train, y_train)

In [46]:
f1_score(RFC1.predict(x_test), y_test, average='weighted')

0.7203922903981894

In [47]:
accuracy_score(RFC1.predict(x_test), y_test)

0.7059585492227979

Модель после Feature Engineering выдала лучший результат по обоим метрикам чем до, что можно считать успехом 

### 3.2.4 Обучение модели на полных данных

Обучение модели на полных данных позволит получить лучшую точность модели на новых данных

In [50]:
RFC = RandomForestClassifier(random_state=1).fit(data.drop(columns='Должность_адресата'), data['Должность_адресата'])

In [54]:
pickle.dump(RFC1, open('model.pkl', 'wb'))

In [53]:
pickle.dump(attrs_to_del, open('attrs_to_delS3.pkl', 'wb'))